<a href="https://colab.research.google.com/github/Syechkyal/LRT-Satification-sentiment-analysis/blob/main/Kelompok1DGTSBigDataUsingPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TUGAS AKHIR PROGRAM FGA DIGITALENTS KELOMPOK 1** #
1. Septika Sari 01123956060-8
2. Kamila Amaliah Putri 01123956060-13
3. Kharisma jhorgi 01123956061-15
4. Syechky Al Qodrin Aruda 01123956081-7
5. Arya Anugrah Putra 01123956061-13

# **Analisa Sentiment Kepuasan Pelanggan LRT menggunakan Metode Multinomial Naive Bayes** 

**Dataset**

Pada percobaan kali ini, kami mendapatkan data set  diambil dari Twitter. Labeling pada tweet tersebut dibagi menjadi kalimat "positif" atau "negatif".

Contoh Labelling:
*   Positive: 
"Kereen nih lihat dgn mata kepala sendiri Kota Wong Kito Galo di Palembang lagi on Progres LRT nya #KEJORApic.twitter.com/qLaxIVDQRm”

* Negative: 
"Light Rail Transit atau LRT Palembang yang sedianya dibangun untuk menyambut acara Asian Games 2018 sudah siap diresmikan oleh Presiden Joko Widodo pada pertengahan Juli 2018 mendatang. https://goo.gl/oUo5JV  #Reservasicom #YukBrangkatpic.twitter.com/mkMYAbaBIL"

Link ke Dataset : https://github.com/apriliaaachi/ProgramSkripsi/blob/master/data/KicauanBerlabel.txt

Link kamus KAMUS_ALAY : https://drive.google.com/file/d/1gLL0me_AVUj_2vWEb3M7lCJfKRpaFWvM/view?usp=sharing

# **Import Library**

**Menghubungkan ke Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
import string

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics

from sklearn.metrics import accuracy_score, classification_report

!pip install Sastrawi
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 35.7 MB/s 


# **Read File dan Convert ke Dataframe**

In [ ]:
df = pd.read_excel("drive/MyDrive/Colab Notebooks/dataset.xlsx")
 
df

,Label,Tweet
0,1,Kereen nih lihat dgn mata kepala sendiri Kota ...
1,1,LRT salah satu efek positif Asian Games 2018 d...
2,1,"Salah satu dampak positif pembangunan LRT, men..."
3,1,"Selain itu, pembangunan LRT dianggap sebagai s..."
4,1,Sumsel Provinsi satu2ny yg memiliki transporta...
...,...,...
533,0,"Terbukti LRT Palembang merugi, dan ada yg berm..."
534,0,Waduuuuh LRT PALEMBANG MERUGI ADA LAGI DI @tvO...
535,0,"Terus Merugi, LRT Palembang Bebani Keuangan Ne..."
536,0,LRT Palembang merugi.. Listrik menunggak.. @M...


# **Preprocessing**


**Menghapus Link URL**

In [ ]:
#Delete link
df['Tweet'] = df['Tweet'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
# df['text'] = df['text'].replace("\n2", " ")


In [ ]:
for text in df['Tweet']:
    print(text)

Kereen nih lihat dgn mata kepala sendiri Kota Wong Kito Galo di Palembang lagi on Progres LRT nya #KEJORApic.twitter.com/qLaxIVDQRm
LRT salah satu efek positif Asian Games 2018 di Palembang... yukkk rame2 sukseskan Asian Games #AlexNoerdin #SatuSriwijayaSatuIndonesia
Salah satu dampak positif pembangunan LRT, menuju masyarakat Sumsel yang lebih modern. #AlexNoerdin"
Selain itu, pembangunan LRT dianggap sebagai solusi kepadatan kendaraan yang berlalu lalang di kota Palembang. #AlexNoerdin"
Sumsel Provinsi satu2ny yg memiliki transportasi masa LRT, mskipun Jkt sdh mulai dluan, ttp Sumsel yg bs sukses pmbangunny. #AlexNoerdin"
Mantab.... Sumsel mendunia dgn transportasi Modern LRT #AlexNoerdin
LRT. akn menjadikan Palembang lebih Modern.. LRT Pertama di Indonesia selesai 2018. #AlexNoerdin @kompascompic.twitter.com/v6wVgTxaW4
LRT jadikan Palembang kota termodern pertama di Indonesia. Budi Karya @kemenhub151 pembangunan LRT Palembang sesuai harapan #AlexNoerdinpic.twitter.com/0PuPW9Qq0h
War

**Mengubah Text Menjadi Lower Case, Menghapus Punctuation, dan Menghapus Angka**

In [ ]:
def clean_tweet(tweet):
  
    '''Make tweet lowercase, remove punctuation and remove words containing numbers. remove newline'''
    tweet = str(tweet)
    tweet = tweet.lower()
    tweet = re.sub('[%s]' % re.escape(string.punctuation.replace('?', '')), '', tweet)
    tweet = re.sub('\w*\d\w*', '', tweet)
    tweet = re.sub('[‘’“”…]', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\r', '', tweet)
    tweet = tweet.replace('?', ' ?')
    tweet = tweet.replace('\d+', '')
    tweet = re.sub('[.;:!\'?,\"()\[\]*~]', '', tweet)
    tweet = re.sub('(<br\s*/><br\s*/>)|(\-)|(\/)', '', tweet)
    return tweet

**Memanggil Fungsi Clean Tweet**



```
# This is formatted as code
```



In [ ]:
df['Tweet'] = df['Tweet'].apply(clean_tweet)

df

,Label,Tweet
0,1,kereen nih lihat dgn mata kepala sendiri kota ...
1,1,lrt salah satu efek positif asian games di pa...
2,1,salah satu dampak positif pembangunan lrt menu...
3,1,selain itu pembangunan lrt dianggap sebagai so...
4,1,sumsel provinsi yg memiliki transportasi masa...
...,...,...
533,0,terbukti lrt palembang merugi dan ada yg berma...
534,0,waduuuuh lrt palembang merugi ada lagi di tvon...
535,0,terus merugi lrt palembang bebani keuangan neg...
536,0,lrt palembang merugi listrik menunggak mbahuy...


In [ ]:
for text in df['Tweet']:
    print(text)

kereen nih lihat dgn mata kepala sendiri kota wong kito galo di palembang lagi on progres lrt nya kejorapictwittercomqlaxivdqrm
lrt salah satu efek positif asian games  di palembang yukkk  sukseskan asian games alexnoerdin satusriwijayasatuindonesia
salah satu dampak positif pembangunan lrt menuju masyarakat sumsel yang lebih modern alexnoerdin
selain itu pembangunan lrt dianggap sebagai solusi kepadatan kendaraan yang berlalu lalang di kota palembang alexnoerdin
sumsel provinsi  yg memiliki transportasi masa lrt mskipun jkt sdh mulai dluan ttp sumsel yg bs sukses pmbangunny alexnoerdin
mantab sumsel mendunia dgn transportasi modern lrt alexnoerdin
lrt akn menjadikan palembang lebih modern lrt pertama di indonesia selesai  alexnoerdin 
lrt jadikan palembang kota termodern pertama di indonesia budi karya  pembangunan lrt palembang sesuai harapan 
warga sumsel patut takjub dgn progres pembangunan lrt sampai hari ini terimakasih alexnoerdin atas kerja kerasnya dlm membangun sumsel 
lrt ba

**Melakukan Word Normalization untuk Mengubah Kata Tidak Baku Menjadi Baku Berdasarkan Dictionary Kamus Alay**

In [ ]:
# alay = pd.read_csv('new_kamusalay.csv')
#alay = pd.read_csv('drive/MyDrive/Colab Notebooks/new_kamusalay.csv')
alay_dict = pd.read_csv('drive/MyDrive/Colab Notebooks//new_kamusalay.csv',ncoding='latin-1', header=None)\
                 .set_index(0)[1].to_dict() e


def normalize_alay(text):
    return ' '.join([alay_dict[word] if word in alay_dict else word for word in text.split(' ')])

df['Tweet'] = df['Tweet'].apply(normalize_alay)
df

,Label,Tweet
0,1,keren ini lihat dengan mata kepala sendiri kot...
1,1,light rail transit salah satu efek positif asi...
2,1,salah satu dampak positif pembangunan light ra...
3,1,selain itu pembangunan light rail transit dian...
4,1,sumatra selatan provinsi yang memiliki transp...
...,...,...
533,0,terbukti light rail transit palembang merugi d...
534,0,waduuuuh light rail transit palembang merugi a...
535,0,terus merugi light rail transit palembang beba...
536,0,light rail transit palembang merugi listrik me...


In [ ]:
for text in df['Tweet']:
    print(text)

keren ini lihat dengan mata kepala sendiri kota orang kito galau di palembang lagi on progres light rail transit nya kejorapictwittercomqlaxivdqrm
light rail transit salah satu efek positif asian games  di palembang yukkk  sukseskan asian games alexnoerdin satusriwijayasatuindonesia
salah satu dampak positif pembangunan light rail transit menuju masyarakat sumatra selatan yang lebih modern alexnoerdin
selain itu pembangunan light rail transit dianggap sebagai solusi kepadatan kendaraan yang berlalu lalang di kota palembang alexnoerdin
sumatra selatan provinsi  yang memiliki transportasi masa light rail transit meskipun jakarta sudah mulai duluan tetap sumatra selatan yang bisa sukses pmbangunny alexnoerdin
mantap sumatra selatan mendunia dengan transportasi modern light rail transit alexnoerdin
light rail transit akan menjadikan palembang lebih modern light rail transit pertama di indonesia selesai  alexnoerdin 
light rail transit jadikan palembang kota termodern pertama di indonesia b

**Melakukan Proses Stemming untuk Mengubah Suatu Kata Kedalam Bentuk Akar Katanya Menggunakan Library Sastrawi**

In [ ]:
stemmer = StemmerFactory()
stemmer = stemmer.create_stemmer()
for idx,text in df['Tweet'].iteritems():
  text = stemmer.stem(text)
  df.at[idx,'Tweet'] = text

df['Tweet']

0      keren ini lihat dengan mata kepala sendiri kot...
1      light rail transit salah satu efek positif asi...
2      salah satu dampak positif bangun light rail tr...
3      selain itu bangun light rail transit anggap ba...
4      sumatra selatan provinsi yang milik transporta...
                             ...                        
533    bukti light rail transit palembang rugi dan ad...
534    waduuuuh light rail transit palembang rugi ada...
535    terus rugi light rail transit palembang beban ...
536    light rail transit palembang rugi listrik tung...
537    kini bukti rugi light rail transit palembang b...
Name: Tweet, Length: 538, dtype: object

In [ ]:
for text in df['Tweet']:
    print(text)

keren ini lihat dengan mata kepala sendiri kota orang kito galau di palembang lagi on progres light rail transit nya kejorapictwittercomqlaxivdqrm
light rail transit salah satu efek positif asi games di palembang yukkk sukses asi games alexnoerdin satusriwijayasatuindonesia
salah satu dampak positif bangun light rail transit tuju masyarakat sumatra selatan yang lebih modern alexnoerdin
selain itu bangun light rail transit anggap bagai solusi padat kendara yang lalu lalang di kota palembang alexnoerdin
sumatra selatan provinsi yang milik transportasi masa light rail transit meski jakarta sudah mulai duluan tetap sumatra selatan yang bisa sukses pmbangunny alexnoerdin
mantap sumatra selatan dunia dengan transportasi modern light rail transit alexnoerdin
light rail transit akan jadi palembang lebih modern light rail transit pertama di indonesia selesai alexnoerdin
light rail transit jadi palembang kota modern pertama di indonesia budi karya bangun light rail transit palembang sesuai harap

**Menyimpan Hasil Pre-Processing**

In [ ]:
df.to_csv('drive/MyDrive/Colab Notebooks/preprocessed_sentimen.csv', index = False)

**Menampilkan Hasil Pre-Processing**

In [ ]:
df = pd.read_csv("drive/MyDrive/Colab Notebooks/preprocessed_sentimen.csv")
df

,Label,Tweet
0,1,keren ini lihat dengan mata kepala sendiri kot...
1,1,light rail transit salah satu efek positif asi...
2,1,salah satu dampak positif bangun light rail tr...
3,1,selain itu bangun light rail transit anggap ba...
4,1,sumatra selatan provinsi yang milik transporta...
...,...,...
533,0,bukti light rail transit palembang rugi dan ad...
534,0,waduuuuh light rail transit palembang rugi ada...
535,0,terus rugi light rail transit palembang beban ...
536,0,light rail transit palembang rugi listrik tung...


# **Data Splitting**

**Melakukan Data Splitting dengan Rasio 70 % Data Training:30 % Data Testing** 

In [ ]:
y_train, y_test = train_test_split(df, test_size=0.3, random_state=42)

**append masing - masing data yang sudah di split ke data clean yang sudah di preprocess sebelumnya.**

Di Term Frequency (TF), kami tinggal menghitung jumlah kata yang muncul di setiap dokumen. Masalah utama dengan Term Frequency ini adalah memberikan bobot lebih pada dokumen yang lebih panjang. 

In [ ]:
train_tweets =[]
for tweet in y_train.Tweet:
    train_tweets.append(tweet)
    
test_tweets =[]
for tweet in y_test.Tweet:
    test_tweets.append(tweet)

Melakukan vektorisasi data menjadi matrix dengan TF-IDF model, kemudian Melakukan fit ke masing - masing data train dan test.

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_tweets)
X_test_tfidf = vectorizer.transform(test_tweets)

**Mengetahui Hasil TF-IDF per Kata yang Dihasilkan**

In [ ]:
print(type(X_train_tfidf))
print(X_train_tfidf.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(376, 1223)


**Menampilkan DataFrame dari hasil TF-IDF**

In [ ]:
# let's look at the dataframe
X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names())
X_train_tfidf

NameError: ignored

**Training data menggunakan Multinomial Naive Bayes, Kemudian Menampilkan Akurasi dari Data Testing**

In [ ]:
# Model Generation Using Multinomial Naive Bayes
clf_tfidf_MNB = MultinomialNB().fit(X_train_tfidf, y_train.Label)
predicted= clf_tfidf_MNB.predict(X_test_tfidf)
print("MultinomialNB Accuracy:",metrics.accuracy_score(predicted, y_test.Label)*100)

MultinomialNB Accuracy: 87.65432098765432


**Melakukan Predict Terhadap Data Uji menggunakan Model Multinomial Naive Bayes**

In [ ]:
clf_tfidf_MNB = MultinomialNB().fit(X_train_tfidf, y_train.Label)
predicted= clf_tfidf_MNB.predict(X_test_tfidf)

**Menampilkan Hasil Prediksi Data Uji**

In [ ]:
clf_tfidf_MNB_predtest = pd.DataFrame({'Tweet' : y_test.Tweet,
                            'Sentiment Predicted' : predicted})
clf_tfidf_MNB_predtest[:30:3]

,Tweet,Sentiment Predicted
485,masalah light rail transit palembang tidak ben...,0
86,light rail transit palembang sudah operasi lig...,0
511,light rail transit palembang sepi minat dari s...,1
428,light rail transit sepi tumpang pasca helat as...,0
406,baru dan mogok lagilagi light rail transit pal...,0
524,saya tinggal di palembang sampai sekarang saya...,0
425,light rail transit palembang memang sepi minat...,0
377,naik light rail transit ngabisin uang tunggu l...,0
204,naik light rail transit bisa lihat indah kota ...,1
354,stasiun light rail transit palembang proyek tr...,0


**Kami melakukan percobaan predict terhadap sentiment, dan dapat dilihat dari hasil sampling 10 data di atas bahwa model Multinomial Naive Bayes memiliki hasil predict yang hampir semua nya benar.**

**Pembagian Job Desc**
1. Syechky Al Qodrin Aruda 01123956081-7
 
  - Mencari Dataset
 
  - Mempresentasikan Program
 - Membuat program untuk memprediksi dan menampilkan hasil klasifikasi pada data uji 
 
2. Kamila Amaliah Putri 01123956060-13
 - Mengkonversi dataset dari file txt ke csv dan meread file ke google colab

 - Membuat program untuk menghitung hasil TF-IDF per kata yang dihasilkan

 - Mencari dan mengimport library apa saja yang dibutuhkan pada program 


3. Septika Sari 01123956060-8
 - Membuat program untuk text cleaning (Menghapus punctuation dan numerik pada kata)

  - Membuat program untuk Word Normalization menggunakan kamus alay 
  - Mencari kamus kata untuk Word Normalization 


4. Kharisma jhorgi 01123956061-15
 - Membuat program untuk Stemming kata 
 - Membuat program untuk training data menggunakan Multinomial Naive Bayes
 - Membuat Grup whatsapp Kelompok
 - Mengkoordinir Kelompok 1

5. Arya Anugrah Putra 01123956061-13
  - Membuat program untuk Case Folding 
 -Membuat program untuk text cleaning (Menghapus link URL) 
  - Membuat program untuk splitting data train dan test


